## 多继承

In [4]:
class A():
    pass 

In [1]:
class A:
    pass

In [2]:
class A(object):
    pass

上面三个类在py3中是完全等价的，所有未声明类的都继承自object类；

In [5]:
class Base:
    pass

In [6]:
class Sub(Base):
    pass

In [8]:
class Sub(Base,object):
    pass

In [9]:
class Base2:
    pass 

In [11]:
class Sub2(Base,Base2):  # 多继承，在继承列表里存在多个类的时候表示多继承；
    pass

多继承会把继承列表里的所有公有成员都继承过来；

同名的时候怎么办？

In [1]:
class A:
    def method(self):
        print('method of A')

In [11]:
class B:
    def method(self):
        print('method of B')

In [3]:
class C(A,B):
    pass 

In [6]:
c = C()

In [7]:
c.method()

method of A


In [14]:
class D(B,A):
    pass

In [15]:
d = D()

In [16]:
d.method()

method of B


## 顺序查找

In [17]:
class E(A):
    def method(self):
        print('method of E')

In [18]:
class F(A,E):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases E, A

In [19]:
class G(E,A):
    pass

In [20]:
g = G()

In [21]:
g.method()

method of E


## MRO: 方法查找顺序

* 本地优先：自己定义或重写的方法优先，按照继承列表，从左到右查找；
* 单调性：所有子类，也要满足查找顺序；

In [22]:
A.__mro__

(__main__.A, object)

In [23]:
E.__mro__

(__main__.E, __main__.A, object)

In [24]:
G.__mro__

(__main__.G, __main__.E, __main__.A, object)

```python 
class F(A,E):
    pass
```

(F,A,E,object)   # 不满足E的单调性；

### C3算法

python通过C3算法来确定是否满足mro的两个原则；

```python 
class B(O) -> [B,O]

class B(A1,A2,....,An) -> [B] + merge(mro(A1),mro(A2),...,mro(An),[A1,A2,...An,O])
```

merge的步骤

* 遍历列表；
* 看第一个列表的首元素：
  - 它在其他列表里也是首元素
  - 它在其他列表里不存在；
* 满足以上两种情况，移除，合并到MRO；
* 不满足，抛出异常；

```
mro(G) -> [G] + merge(mro(E),mro(A))
       -> [G] + merge([E,A,O],[A,O],[E,A,O])
       -> [G,E] + merge([A,O],[A,O],[A,O])
       -> [G,E,A] + merge([O],[O],[O])
       -> [G,E,A,O]
```

In [28]:
G.__mro__

(__main__.G, __main__.E, __main__.A, object)

```
mro(F) -> [F] + merge(mro(A),mro(E),[A,E,O])
       -> [F] + merge([A,O],[E,A,O]，[A,E,O])
       -> raise Exception
```

当一个类定义的时候，解释器会执行C3算法来确定mro，如果C3算法抛出异常，此类不能定义；

应该尽量避免多继承

In [31]:
class A:
    pass

class B(A):
    pass

class C(A):
    pass

class D(B,A):
    pass

class E(C,A):
    pass

class F(E):
    pass 

class G(F,D):
    pass

多继承会对程序员的心智负担造成非常大的压力；

python是解释执行的，一段代码只有执行到的时候，才知道有没有错。

多继承是一剂毒药，有的语言(java)明确禁止多继承；

类装饰器：

    设置一些类变量；
    
    可以给类增加一些方法；  # 但是不够优雅；

## Mixin

如何以AOP(面向方面)的方式给类增加方法？

       Document

    Word   Excel

printableWord printableExcel

PrintToPrinterWord  


In [32]:
def print_to_printer(cls):
    def print(self):
        pass 
    cls.print_to_printer = print
    return cls 

如果一个类，不能冬天太的增加成员的时候，这种方法行不同；

In [35]:
class Document:
    def __init__(self,content):
        self.content = content

In [36]:
class Word(Document):
    def __init__(self,content):
        super().__init__('word: {}'.format(content))

In [37]:
class Excel(Document):
    def __init__(self,content):
        super().__init__('excel:{}'.format(content))

In [46]:
def printable(cls):
    def _print(self):
        print(self.content)
    cls.print = _print
    return cls

In [47]:
@printable
class PrintableWord(Word):
    def __init__(self,content):
        super().__init__(content)

In [48]:
pw = PrintableWord('abc')

In [49]:
pw.print()

word: abc


In [50]:
def print_to_monitor(cls):
    def _print(self):
        print('Monitor:P: {}'.format(self.content))
    cls.print = _print
    return cls

In [51]:
@print_to_monitor
class PrintMonitorWord(Word):
    def __init__(self,content):
        super().__init__(content)

In [52]:
pmw = PrintMonitorWord('abc')

In [53]:
pmw.print()

Monitor:P: word: abc


---

In [62]:
class PrintableMixin:
    def print(self):
        result = 'P: {}'.format(self.content)
        print(result)
        return result

In [63]:
class PrintableWord(PrintableMixin,Word):
    def __init__(self,content):
        super().__init__(content)

In [64]:
pw = PrintableWord('abc')

In [65]:
pw.print()

P: word: abc


'P: word: abc'

In [70]:
class PrintToMonitorMixin(PrintableMixin):
    def print(self):
        print('Monitor:{}'.format(super().print()))

In [71]:
class PrintToMonitorWord(PrintToMonitorMixin,Word):
    def __init__(self,content):
        super().__init__(content)    

In [72]:
pw = PrintToMonitorWord('abc')

In [73]:
pw.print()

P: word: abc
Monitor:P: word: abc


Mixin其实也是一种组合的方式；

通常来说，组合优于继承；

Mixin 类的限制：
* Mixin 类不应该有初始化方法；
* Mixin类通常不能独立工作；
* Mixin类的祖先也应该是Mixin类；

通常情况下，Mixin类总在继承列表的第一位；

类： 无法修改

扩展一些方法

扩展的方法，写到一个Mixin类里；

写一个类，继承Mixin和无法修改的类；

新的类就具有了要扩展的方法了。

Mixin是依靠多继承来实现的，除了使用Mixin的时候，尽量避免使用多继承；

---
---

## 魔术方法/专有方法

In [74]:
class A:
    pass

In [77]:
dir(A)   # 用内置函数dir()可以得到类A的所有公有成员；

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

### 属性：

In [78]:
a = A()

In [79]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

### 类属性：

In [80]:
A.__name__

'A'

In [81]:
A.__module__

'__main__'

In [83]:
A.__doc__   # 文档字符串

In [84]:
A.__class__

type

### 实例属性：

In [85]:
a.__name__   # 实例没有name

AttributeError: 'A' object has no attribute '__name__'

In [86]:
a.__doc__

In [87]:
a.__module__

'__main__'

In [88]:
a.__class__

__main__.A

In [91]:
a.__class__.__name__   # 得到实例的类名；

'A'

In [93]:
a.__dict__  # 实例的所有属性，都保存在__dict__里

{}

In [94]:
a.xxx = 3

In [95]:
a.__dict__

{'xxx': 3}

In [96]:
a.__dict__['xxx'] = 5

In [97]:
a.xxx

5

In [99]:
a.__dict__[set(1,2,3)] = 5    # set(1,2,3) 对于字典来说是可以做key的，但对于实例不行；

TypeError: set expected at most 1 arguments, got 3

In [100]:
a.__dict__['x x x'] = 5

In [101]:
a.__dict__

{'x x x': 5, 'xxx': 5}

In [102]:
a.__dict__['x x x']

5

In [103]:
a.x x x 

SyntaxError: invalid syntax (<ipython-input-103-7b9f7a82be6c>, line 1)

In [104]:
a.__dir__

<function A.__dir__>

In [107]:
a.__dir__()   # 得到实例的所有成员，包括方法和属性，dir()函数调用的是__dir__()

['xxx',
 '__subclasshook__',
 '__delattr__',
 '__sizeof__',
 '__format__',
 '__gt__',
 '__le__',
 '__repr__',
 '__reduce__',
 '__ge__',
 '__getattribute__',
 '__class__',
 'x x x',
 '__weakref__',
 '__setattr__',
 '__lt__',
 '__module__',
 '__new__',
 '__init__',
 '__hash__',
 '__eq__',
 '__ne__',
 '__dir__',
 '__str__',
 '__reduce_ex__',
 '__dict__',
 '__doc__']

In [110]:
dir(a) == a.__dir__()  # dir(a)经过了排序

False

In [109]:
dir(a).sort() == a.__dir__().sort()   

True

### 分类：

* 创建/销毁
* 运算符重载
* hash
* bool 
* 可视化
* 反射
* 上下文管理
* 大小
* 描述器
* 杂项

### 运算符重载

In [116]:
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def add(self,other):
        return Point(self.x + other.x,self.y + other.y)        

In [117]:
a = Point(0,0)

In [118]:
b = Point(3,5)

In [119]:
a + b

TypeError: unsupported operand type(s) for +: 'Point' and 'Point'

In [120]:
a.add(b)

In [121]:
c = a.add(b)

In [122]:
c

In [123]:
c.x

3

In [124]:
c.y

5

In [126]:
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __add__(self,other):
        return Point(self.x + other.x,self.y + other.y)   

In [127]:
a = Point(0,0)
b = Point(3,5)

In [128]:
a + b

In [129]:
c = a + b

In [131]:
c.x

3

In [132]:
c.y

5

In [133]:
c += Point(4,6)

In [134]:
c.x

7

In [135]:
c.y

11

**算数运算**

通过int可以找到大多数的算数运算符重载方法；

```python 
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __add__(self,other):
        return Point(self.x + other.x,self.y + other.y)

    def __sub__(self, other):
        return Point(self.x - other.x,self.y - other.y)


a = Point(0,0)
b = Point(3,5)
c = a + b

# print(c.x,c.y)
#
# c += Point(4,6)
# print(c.x,c.y)

d = a - b
print(d.x,d.y)

d -= Point(-5,-8)
print(d.x,d.y)
```

### 位运算
### 比较运算
### 成员运算

身份运算和赋值运算，无法重载

不要过度的使用运算符重载